# Modèle avancé - Transformer

Nous avons vu qu'une limite du modèle XGBoost est le fait qu'il ne prenne pas en compte la **dimension séquentielle** des actions au cours du temps. 

Pour capturer les relations d'une séquence, différents modèles d'apprentissage profond ont vu le jour. Jusque la fin des années 2010, les méthodes de traitement de données séquentielles se basaient principalement sur de la récurrence (**réseaux de neurones récurrents**) ou de la convolution (**réseaux de neurones convolutifs**). Elles étaient ainsi couplées à un mécanisme d'attention, qui permet de déterminer l'importance de chaque élément de la séquence relativement à ses autres éléments.

En 2017, le papier de recherche "Attention Is All You Need" (A. Vaswani, N. Shazeer, N. Parmar) introduit l'architecture **Transformer**, et révolutionne le traitement de données séquentielles. L'idée est de se baser exclusivement sur le mécanisme d'attention, ce qui permet une meilleure parallélisation des calculs, et un temps d'entraînement plus court.

Dans ce Notebook, nous proposons une implémentation d'une architecture Transformer pour résoudre la prédiction des utilisateurs.

## Sommaire

- [Lecture des données](#lecture-des-données)
- [Extraction des caractéristiques temporelles](#extraction-de-caractéristiques-temporelles)
- [Tokenisation](#tokenisation)
- [Sélection du modèle](#sélection-du-modèle)
  - [Premier modèle : Transformer à un seul contexte](#premier-modèle--transformer-à-un-seul-contexte)
  - [Deuxième stratégie : Transformer multi-contexte](#deuxième-stratégie--transformer-multi-contexte) 
- [Evaluation](#prédiction)

## Lecture des données

In [ ]:
import pandas as pd
from model.reader import reader

df = reader('train.csv')

In [ ]:
TARGET = df.iloc[:, 0]
browsers = df.iloc[:, 1]
sequence_lengths = df.iloc[:, 2]
actions = df.iloc[:, 3:]

## Extraction de caractéristiques temporelles

A l'aide des séquences d'actions et de la durée totale de la séquence, on extrait deux caractéristiques temporelles :
- la durée de la session (en secondes) T
- la vitesse d'action (en nombre d'actions par seconde), calculé selon n_actions / T

In [ ]:
from model.time_features import bucketize_time_features, compute_time_features

time_features = compute_time_features(actions, sequence_lengths)
time_features = bucketize_time_features(time_features)

duration_tokens = list(time_features['duration_bucket'])
speed_tokens = list(time_features['speed_bucket'])

## Tokenisation

In [ ]:
from model.tokenizer import tokenize_action_sequence, tokenize_browser_data, tokenize_username_data

username_tokens, username_to_idx = tokenize_username_data(TARGET)
action_tokens, action_to_idx = tokenize_action_sequence(actions)
browser_tokens, browser_to_idx = tokenize_browser_data(browsers)

## Sélection du modèle

Tout d'abord, prenons en charge CUDA pour entraîner le modèle sur un GPU NVIDIA T4.

In [ ]:
import torch

# GPU setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    
    # Enable memory optimization
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False

L'entraînement de notre Transformer peut être résumé en quatre étapes :
- On tokenize le texte, on ajoute des encodages de position et on préfixe par un ou plusieurs tokens de contexte pour conditionner la suite.
- Les couches d’auto-attention + MLP calculent des représentations contextuelles, avec masquage pour ne pas “voir” le futur en auto-régressif.
- La tête de sortie prédit la distribution du prochain token, puis on compare au vrai token via une perte d’entropie croisée.
- On rétro-propage l’erreur et on met à jour les poids sur de grands époques jusqu’à convergence.

### Premier modèle : Transformer à un seul contexte

La première architecture de Transformer que nous décidons d'entraînµer est un modèle à tête qui prend uniquement le token de contexte "Browser".

Nous commençons par créer le modèle :

In [ ]:
from model.transformer import create_model, train_model

model = create_model(
    vocab_size=len(action_to_idx),
    n_usernames=len(username_to_idx),
    n_browsers=len(browser_to_idx),
    d_model=256,        
    n_heads=8,          
    n_layers=6,         
    d_ff=512,
    max_seq_len=500,
    dropout=0.1
)

# Move model to GPU
model = model.to(device)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

On réalise l'entraînement sur un GPU T4.

In [ ]:
# Split data (simple 80/20 split)
split_idx = int(0.8 * len(df))
train_data = (
    action_tokens[:split_idx],
    username_tokens[:split_idx],
    browser_tokens[:split_idx]
)
val_data = (
    action_tokens[split_idx:],
    username_tokens[split_idx:],
    browser_tokens[split_idx:]
)
 
train_model(
    model,
    train_data=train_data,
    val_data=val_data,
    epochs=100,
    batch_size=32,      
    max_seq_len=500,
    device=device
)

### Deuxième stratégie : Transformer multi-contexte

Pour prendre en compte la dimension temporelle de nos données, nous décidons d'augmenter le contexte de deux tokens supplémentaires, extraits des jalons d
temporels : la vitesse d'action de l'utilisateur et la durée de la session.

Les têtes de lecture du modèle Transformer vont donc parcourir les séquences avec 3 tokens persistants en début de fenêtre : "browser", "duration_bucket", "speed_bucket"

In [ ]:
from model.transformer_extended_context import create_model, train_model

discrete_contexts = {
    'browser': 4,
    'duration_bucket': 8,
    'speed_bucket': 8
}

# Create optimized model for T4 GPU
model = create_model(
    vocab_size=len(action_to_idx),
    n_usernames=len(username_to_idx),
    discrete_contexts=discrete_contexts,
    d_model=256,       
    n_heads=8,          
    n_layers=6,        
    d_ff=512,
    max_seq_len=100,
    dropout=0.1
)

# Move model to GPU
model = model.to(device)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
split_idx = int(0.8 * len(df))
train_data = (
    action_tokens[:split_idx],
    username_tokens[:split_idx],
    browser_tokens[:split_idx],
    duration_tokens[:split_idx],
    speed_tokens[:split_idx]
)
val_data = (
    action_tokens[split_idx:],
    username_tokens[split_idx:],
    browser_tokens[split_idx:],
    duration_tokens[split_idx:],
    speed_tokens[split_idx:]
)
 
# Train with memory-efficient parameters
train_model(
    model,
    train_data=train_data,
    val_data=val_data,
    epochs=100,
    batch_size=32,      # Larger batch size for GPU
    max_seq_len=500,
    device=device
)

### Troisième stratégie : Transformer avec Embeddings

On crée des embeddings pour "browser", "duration", "speed", et aussi pour les statistiques de distribution des tokens.

In [ ]:
from model.transformer_3 import create_model, train_model

discrete_contexts = {
    'browser': 4,
    'duration_bucket': 8,
    'speed_bucket': 8
}

# Create optimized model for T4 GPU
model = create_model(
    vocab_size=len(action_to_idx),
    n_usernames=len(username_to_idx),
    discrete_contexts=discrete_contexts,
    d_model=256,       
    n_heads=8,          
    n_layers=6,        
    d_ff=512,
    max_seq_len=2000,
    dropout=0.1,
    context_as_features=True,
    use_token_statistics=True,
    token_stats_top_k=10
)

# Move model to GPU
model = model.to(device)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

split_idx = int(0.8 * len(df))
train_data = (
    action_tokens[:split_idx],
    username_tokens[:split_idx],
    browser_tokens[:split_idx],
    duration_tokens[:split_idx],
    speed_tokens[:split_idx]
)
val_data = (
    action_tokens[split_idx:],
    username_tokens[split_idx:],
    browser_tokens[split_idx:],
    duration_tokens[split_idx:],
    speed_tokens[split_idx:]
)
 
# Train with memory-efficient parameters
train_model(
    model,
    train_data=train_data,
    val_data=val_data,
    epochs=100,
    batch_size=32,     
    max_seq_len=2000,
    device=device
)

## Prédiction

Use test.csv

In [ ]:
test_df = reader('test.csv', training=False)

browsers = test_df.iloc[:, 0]
sequence_lengths = test_df.iloc[:, 1]
actions = test_df.iloc[:, 2:]

action_tokens, _ = tokenize_action_sequence(actions=actions, existing_token_to_idx=action_to_idx, training=False)
browser_tokens, _ = tokenize_browser_data(browsers=browsers, existing_browser_to_idx=browser_to_idx, training=False)
idx_to_username = {idx: username for username, idx in username_to_idx.items()}

submission = []

for i in range(len(action_tokens)):
    action_sequence = torch.tensor(action_tokens[i]).to(device)
    browser = torch.tensor(browser_tokens[i]).to(device)
    logits, probs = model.predict_username(action_sequence, browser)
    predicted_username = torch.argmax(logits, dim=-1)
    predicted_idx = predicted_username.item()
    predicted_username_name = idx_to_username[predicted_idx]
    submission.append(predicted_username_name)



In [ ]:
submission

Sauvegarde de la prédiction en CSV

In [ ]:
df_subm = pd.DataFrame({"prediction": submission})
df_subm = df_subm.rename_axis("RowId")
df_subm.index = df_subm.index + 1

df_subm.to_csv("output/submission_5.csv")

## Conclusion

L'entraînement d'un modèle Transformer présente des avantages (capture de l'ordre de la séquence), mais aussi des inconvénients.


Notre jeu de données présente peu d'exemples par utilisateur, ce qui rend l'utilisation du deep learning propice à l'overfitting. En particulier, notre jeu de données présente un déséquilibre de classes, avec un nombre de session d'utilisateur entre 4 et 72. 